# Introduction
## Books
1. Routing TCP/IP vol2 
2. Developping IPv4 Multicast : https://epdf.pub/developing-ip-multicast-networks.html
3. IP Multicast technology : https://www.cisco.com/c/en/us/td/docs/ios/solutions_docs/ip_multicast/White_papers/mcst_ovr.html
4. MC best practices : https://www.cisco.com/c/en/us/products/collateral/ios-nx-os-software/multicast-enterprise/whitepaper_c11-474791.html

## What is Multicast ?
* Multicast is a data transmission to a group of destination simultainusly (i.e. One-to-many communication). 
* As Opposed to:
    1. __Unucast__ : One-to-one
    2. __Broadcast__ : One-to-all
    3. __Anycast__ :One-to-nearest one
    
## Why use Multicast ? 
The main goal of MC is to reduce the load on.
1. __Sending server processing__ : sender sends one copy that gets replicated by the network.  
2. __Network bandwidth resources__ : one copy of the feed is transferred through the netowork adn gets repicated at the interface level. 
3. __Router forwarding processing__ : Only one feed needs to be processed. 
4. __Receiving host processing__ : Hosts only receive packets for the groups they join. 

### why not just use Unicast ?
* Sender generate one packet per destination (__head-end replication__)
* Sender must know addresses of all destinations 
* router muust process packets for all destination 
* bandwidth use is proportional to the number of users, in MC the BW utilization is proportional to the number of senders. 

### Why not Broadcast ? 
* Packets are forwarded on all packets except the receiving one, and can't be routed between multiple different subnets. 
* All users regardless of their need gets packets 
* results inefficient bandwidth utilization 

### How multicast optimises the Bandwidth ? 
* Source generate single data field for all interested recipient 
* source does not know who's recieving 
* Router makes single forwarding decission for all recipients 
* Only one packet is replicated per interface (saves bandwidth) 
* Unwilling hosts dont receive any packets. 

## Disadvantages of Multicast
IP Multicast is UDP only. Problem with __Connectionless__ transmission are,
1. __Best-effort__ delivary : no ACK 
2. __No Congestion avoidence__: no Slow start and QoS
3. __Possible Duplicatation__ : usually only the network is reconverging
4. __Out-of-Order Packets__ : no sequencing 

## How multicast works? 
1. Source application _sends UDP multicast traffic with a group_ destination address. 
2. Ingterested receivers _join group addess_ by signalling its gateway routers 
3. Routers _build a loop-free tree_ from the sender to reciver
4. data feed is _repilicated only for the willing hosts_ 

### Use cases
1. Multimedia (e.g. IPTv, Video-conferencing, VoIP, music podcast )
2. Data Distribution (large scale datacenter replication)
3. Realtime application (stock tickers)

## IPv4 MC components

### 1. Group addressing: 
* L2 and L3 addressing 
* Multicast __group__ is an address, aggreed upon between the sender and receivers for a particular feed. 
* Source sends traffic to the desination address of the group and all willing receivers recieve the feed. 
* traffic is always sent __to__ the gorup __never from__ (i.e. it's __Destination bound__).
* group uses both L2 and L3 addresses
* For __L3 addressing__ it uses class __D__ addresses $224.0.0.0/4 (224.0.0.0 - 239.255.255.255)$ including reserve addresses 
    1. __Link local address__ : `224.0.0.0` - `224.0.0.255`
    2. __Source Specific MC__ : `232.0.0.0` - `232.255.255.255`
    3. __Administratively scoped__ : Private `239.0.0.0` - `239.255.255.255`
* IPv4 MC, __L3 addresses are mapped to a L2 (MAC)__ addresses to forward traffic on the LAN. This allows L2 _switches to forward MC_ intelligently (i.e. not to forward MC as BC).
    1. MC MAC range is : `01-00-5E-00-00-00` to `01-00-5E-7F-FF-FF`
    2. 25 MSBs are fixed and 23 LSBs are mapped from IPv4
    3. To avoid overlap between MC groups, last 23 bits must be unique after mapping 
    4. __Mapping rule__ 
        1. IPv4 Address : $[224-239].X.Y.Z$
        2. Convert second octat into binary and set the MSB as 0 i.e. $(X)_8 = (x_7 x_6 x_5 ... x_0)_2 \rightarrow (0 x_6 x_5 ... x_0)_2=X'$
        3. Convert $X'$ into Hex
        4. Convert $Y$ and $Z$ directly to Hex
    5. e.g. `224.0.0.1` = `01-00-5e-00-00-01`
    6. e.g. `230.255.1.2` = `01-00-5e-7f-01-02`
    6. e.g. `239.127.1.2` = `01-00-5e-7f-01-02` (overlapping with previous example) 


### 2.  Control plane:
* Protocols: IGMP(host-router), PIM, MSDP (router-router), MBGP
* Determinses: who's sending the traffic to what group and who's receiving traffic from what group __(s,G)__, and how the traffic will be forwarded (__the MC Tree__) 
* IGMP is used to signal the gateway router from the host to join a apecific group. (RFC: V1:1112, V2:2236, V3:3337)
* IGMP Versions
    * __IGMPv1__: uses two message types to signal group membership, __Host membership query__ (used by routers to verify if the members are still active, uses a groupwise __idle timer__) and __host membership report__ (used by host to signal joining). This version is replaced by version2
    * __IGMPv2__: Enhances version 1 by adding 4 components __Querier Election__ (if multiple routers on the segment, who sends queries?), __Tunable Timers__ (can control query response timeouts), __Group specific Queries__ (Queries sent to the group address instead of the host) and __Explicit leave__ (speeds up convergence if no other hosts join that group). This version is backward compatible to version 1
    * __IGMPv3__: Uses __Source specific multicast (SSM)__ i.e. v1 and v2 only support group specific joins $(*,G)$ whereas, v3 specify source as well $(S,G)$, this implies the receiver to know about the sender. 
* Once the gateway router $R$ is informed about $(S/*,G)$ by IGMP, $R$ has to inform the network to get traffic from the source, this is done by the Multicast routing protocols i.e. __Protocol Independent Multicast (PIM)__ not MBGP/MSDP 
* __PIM__ is a R-R communication to build a __Loop free tree__ from sender to reciever(s). It is __protocol Independent__ as it doesn't advertise its own topology information, relies on the underlying IGP protocols that build the loop free topology. __Versions__(PIMv1,v2), __Modes__(Sparse, Dense) controlls how the tree will be built. 
    * __PIM Dense Mode__ : considers implicit join i.e. you recive all traffic unless you explicitly step out, this is called _flood and prune_ behaviour. 
    * __PIM Sparse Mode__: consideres explicit join i.e. you won't receive any traffic unless you explicity ask for it. uses __Rendezvous Point (RP)__ to process the join requests. 
    
### 2.  Data Plane:
* Once the MC-tree is built, traffic begins to flow but before forwarding Data-plane checks occur. 
* __Reverse Path forwarding (RPF)__ check - this is a loop prevension mechanism in MC, i.e. was traffic recived on the correct interface ? When a MC packet appears to an interface of a router, _if the incomming interface = outgoing intrface for the source address of the MC packet by as per CEF_ , then it passes RPF check check, otherwise fails and the router drops the packet (MC ingress must be UC egress). Tis reles on the loop free topology built by the unlderlying IGP. 
* MC routing table (MRIB, MFIB) - what interface should the traffic be forwarded to ? AN inteface facing upstream towards the source if called incomming and interfaces facing towards the receivers are the outgoing interface list (OIL). __MC split-horizon__ suggests an incomming interface can't belong to OIL for a given MC. 

# L2 multicast 

* Ethernet supports multicast natively, MC bit in 48 bits MAC address is __Least significant bit of the most significant byte__ e.g. `01:00:CC:CC:CC:CC`. Typically MC is used in L2 for CDP and L2 tunnelling etc. adresses are reserved by IEEE. 
* IPv4 MC addresses are mapped into MC MAC addresses within the range of `01-00-5e-00-00-00`-`01-00-5e-ff-ff-ff` 32 bits are mapped into 23 bits thus overlap occurs. 
* Swithces treat unknown unicast and multicast as broadcast, for VLANs MC traffic is forwarded to __all ports that have a receiver__. 
* __IGMP & PIM Snooping__ : How does the switch know where the MC receivers and routers are? the switch listen to the IGMP reports/leave messages, extracts the group address reported and prune unnessesary multicast traffics. Routers process all multicast traffic, switch listen to the PIM messages (PIM Snooping) and finally discovers the Upstream inteface (to the router) downstream interfaces (to the devices). 
* __Commands__
    * `ip igmp snooping [vlan VLAN-ID]` to enable snooping
    * `ip igmp snooping vlan VLAN-ID static IP-ADDR interface IFACE` for static joining of a port to a group. 
* __IGMP snooping and STP__: STP topology change may signal receiver moving. After a TCN event, switch floods all multicast groups on all ports. The command `ip igmp snooping tcn flood query count COUNT` means, flood until `COUNT` number of query intervals have expired. Disabling flooding tuding TCN using `no ip igmp snooping tcn flood` command. 
* __IGMP Profiles__ : `ip igmp access-group` applies only on L3 interfaces, `ip igmp profile` allowes IGMP access control in L2.
    * profiles are either permit/deny mode
    * __Permit__ mode allows a specifig group and blocks rest, __deny__ mode blocks a specific group and allows rest.
    ```
    ip igmp profile 123
        permit 
            range 239.0.0.0
            range 237.0.0.0 238.0.0.0
            
    interface f0/1
        ip igmp profile 123
    ```
* __IGMP Throttling__: Limits the number of goups an interface can join, new groups are either __denied/replaced__ by old ones. This prevents any buffer-overflow attack. 
```
ip igmp max-groups N
ip igmp max-groups action {deny|replaces}
```


# PIM in sparse mode
Specified standard (RFC 4601), uses __Pull model__ or __explicit join__ i.e. traffic is not flooded until asked for. Uses both Shared Trees (__RPT__) that is rooted at the RP and Shortest path trees (__SPT__) that's rooted at the center. Dense model uses only SPT (legacy), it is more scalable than Desnse mode and a better design choice. 

## Shared vs Source Trees
MC tree determines how traffic is routed from sender to receivers

| Sourced tree | Shared Tree |
|--- |---|
|uses a shortes path from sender to receiver | uses a shortest path from sender to RP, then shortest path from RP to receiver |
|dense mode and sparse mode | sparse mode only |
| | brings more scalability in routing table by eliminating _flood and prune_ beaviour

## PIM sparse mode opeation 
* step1: discover PIM neighbour and elect DR
* step2: discover RP
* step3: inform RP about sources (__pim register__ message)
* step4: inform RP about receivers (__pim join__ message)
* step5: builed sared tree from sender to receiver though RP
* step6: join Shorted path tree
* step7: Leave shared tree
* step8: maintain MC routing table

## Rendezvous Point 
* RP is used as a __Reference point__ for the root of the shared tree
* RP learns about the source from __Unicast__ PIM register message $(S,G)$ and learns about the receivers from the PIM join message $(*,G)$. 
* RP is used to merge the two trees togather. 

### Learning the RP address
Without the RP, sources can't be register nor join can be processesd. All routers must agree on the RP address on a __Per-group__ basis. In case of an invalid RP, Register and Join messages are rejected.RP can be assigned statically or dynamically (using Auto-RP or BSR).

### Step 1: Shared-tree building  process
As the root of all shared trees, the RP must know all source addresses and all receiver addresses. 

|  	| PIM Registration process 	| PIM Join process 	|
|-	|-	|-	|
|  	| When the first-hop router connected to the source hears the traffic, a __Unicast Register__ message is sent to the RP. For multiple first-hop routers, only DR registers. 	| When the last-hop router receives an IGMP-Report/join, a __PIM-Join__ message is genrated up the reverse path tree towards the RP 	|
|  	| If RP accepts this message, it acknowledges with a __Register-STOP__ message and inserts $(S,G)$ into the MC routing table. 	| All the routers in the reverse path install $(*,G)$ and forwards the join hop-by-hop to the RP 	|
|  	| At this point DR and RP know $(S,G)$ 	| At this point the RP and all downstream routers towards the receiver know $(*,G)$ 	|

### Step 2: Merging through RP
Once, the the registraton and joining is done the two trees are __merged__ at the RP
* RP sends a __PIM-join__ message up the reverse path to source.
* all the routers in the reverese path from RP to source install $(*,G)$ with OIL pointing towards RP
* Once $(S,G)$ begins to flow, the tree is built end-to-end through the RP

### Step3: Joining the SPTs
The shared tree is made up of two STPs (From reciever to RP and from RP to source). The shared Tree (RPT) may not be an optimal SPT as routing traffic through RP may not efficient always. To fix this problem The __Last-hop router__...
* joins the SPT to source with $(S,G)$ Join message
* Leaves the RPT by sending $(*,G)$ prune to RP

This can be modefied by `ip pim spt-threshold` command. 

### Step 4: MC routing table maintainence 
* PIM uses __State-refresh__ to ensure that feeds don't timeout, $(*,G)$ Join message is sent to RP or up to SPT to refresh the OIL. 
* __Sparse-Prune__ message can be used to speed up state information timeout if IGMP leave is heard from an end host. 